In [11]:
dayObs = '20250420'
run = "None"
sensor='R10_S10'

In [12]:
from lsst.daf.butler import Butler
from astropy.table import Table
from astropy.time import Time, TimeDelta
import tabulate
import numpy
from find_missing import main

In [13]:
if dayObs=='Today':
    dayObs = (Time.now() - TimeDelta(0.5, format='jd')).iso[:10].replace('-','')

In [14]:
butler = Butler('embargo_new', collections=[
    'LSSTCam/raw/all',
    'LSSTCam/calib/unbounded',
    'LSSTCam/photodiode'
])

In [15]:
exposureList = []

if run == "None":
    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"
else:
    where=f"exposure.science_program='{run}' and instrument='LSSTCam'"    

for record in butler.registry.queryDimensionRecords("exposure", where=where):
    nimage=len(list(butler.registry.queryDatasets('raw',dataId=record.dataId)))
    hasPD=len(list(butler.registry.queryDatasets('photodiode',dataId=record.dataId)))
    exposureList.append([record.id, record, nimage, hasPD])
exposureList.sort(key=lambda x: x[0])
table = Table(rows=[ (
            f'<a href=http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image={record.obs_id} target="_blank">{record.obs_id}</a>',
            record.timespan.begin.iso,
            record.seq_num,
            record.observation_type,
            record.science_program,
            record.observation_reason,
            record.exposure_time,
            record.physical_filter,
            nimage,
            f'<a href=https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/CheckPDTraces?exposure={record.id} target="_blank" >PD</a>' if hasPD else "",
            f'<a href=https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/FitsHeader?expid={record.obs_id}&detector={sensor} target="_blank" >meta</a>',    
                     )
            for [id,record,nimage, hasPD] in exposureList ]
            )

In [16]:
subtable=Table(rows=[  ( record.obs_id, record.seq_num, nimage ) for [id,record,nimage, hasPD] in exposureList ], names=(  'id', 'seqnum', 'nimage'  ) )

# Possible data transfer issue

In [17]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['seqnum'])!=1], subtable[1:][numpy.diff(subtable['seqnum'])!=1]):
    print(a['id'], b['id'])

# KT's check

In [ ]:
main("LSSTCam",dayObs)

dayobs='20250420' seqnum=1 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' seqnum=2 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' seqnum=3 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' seqnum=4 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' seqnum=5 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' seqnum=6 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' seqnum=7 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' seqnum=8 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' seqnum=9 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' seqnum=10 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' seqnum=11 expect_s=197 expect_g=0 found_s=197 found_g=0 ingest_s=197 OK
dayobs='20250420' s

# Changes in number of images ingested

In [ ]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['nimage'])!=0], subtable[1:][numpy.diff(subtable['nimage'])!=0]):
    print(a['id'], a['nimage'], b['id'], b['nimage'])

# Full records

In [ ]:
print(f"The number of exposures is {len(table)}")
tabulate.tabulate(table[::-1], tablefmt='unsafehtml',headers=[
                   'obs_id', 'time', 'seqnum', 'observation_type',
                   'science_program', 'observation_reason','exposure_time',
                   'physical_filter', 'nimage', 'hasPD', 'metadata'
                  ])